In [1]:

import scipy as sp
from scipy.spatial.distance import squareform, pdist

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold

# para testes
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import zero_one_loss, accuracy_score


dfac = pd.read_csv('dados/mfeat-fac', delim_whitespace=True, header=None)
dfou = pd.read_csv('dados/mfeat-fou', delim_whitespace=True, header=None)
dkar = pd.read_csv('dados/mfeat-kar', delim_whitespace=True, header=None)
#dmor = pd.read_csv('dados/mfeat-mor', delim_whitespace=True, header=None)
#dpix = pd.read_csv('dados/mfeat-pix', delim_whitespace=True, header=None)
#dzer = pd.read_csv('dados/mfeat-zer', delim_whitespace=True, header=None)

# Impressão dos dados

In [2]:
# Todos os conjuntos de dados possuem 2000 linhas, 200 instancias por classe (classes de 0-9)

In [3]:
dfac[:100]

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,98,236,531,673,607,647,2,9,3,6,...,474,536,628,632,18,36,8,15,12,13
1,121,193,607,611,585,665,7,9,2,4,...,520,458,570,634,15,32,11,13,15,11
2,115,141,590,605,557,627,12,6,3,3,...,535,498,572,656,20,35,16,14,13,6
3,90,122,627,692,607,642,0,6,4,5,...,576,549,628,621,16,35,7,12,15,9
4,157,167,681,666,587,666,8,6,1,4,...,594,525,568,653,16,35,10,15,13,13
5,128,224,799,690,653,620,16,22,8,9,...,692,529,562,573,24,17,7,10,14,6
6,185,259,575,615,609,673,2,8,5,7,...,518,502,632,698,18,35,16,14,14,14
7,133,173,591,665,594,651,1,7,3,5,...,520,522,567,612,15,34,8,16,11,13
8,206,332,561,588,635,693,3,9,8,7,...,490,473,624,698,19,36,19,14,11,17
9,183,177,583,606,627,676,1,8,1,4,...,538,509,648,709,15,33,14,13,15,12


In [4]:
dfou[:4]

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.065882,0.197312,0.103826,0.270362,0.616078,0.035856,0.424572,0.089701,0.367773,0.037065,...,0.123448,0.113374,0.074343,0.281666,0.067795,0.344182,0.038963,0.394366,0.049971,0.344871
1,0.049142,0.175971,0.105515,0.227095,0.599280,0.041217,0.431078,0.096801,0.326739,0.059661,...,0.045310,0.069337,0.045386,0.254264,0.045447,0.335659,0.021719,0.445277,0.083978,0.354092
2,0.034172,0.227649,0.108766,0.127697,0.612494,0.056554,0.470639,0.041903,0.324267,0.044569,...,0.019858,0.218842,0.041087,0.360464,0.047154,0.377408,0.052099,0.445029,0.071234,0.261465
3,0.062336,0.217979,0.080243,0.289592,0.546316,0.045779,0.425545,0.022841,0.331454,0.119052,...,0.083995,0.085479,0.087658,0.151880,0.075950,0.293462,0.022675,0.408291,0.063010,0.401376


In [5]:
dkar[:10]

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-10.297008,-11.666789,11.560669,-2.081316,4.044656,4.086815,-2.558072,-8.476935,2.138135,3.503082,...,1.078083,0.921927,0.496387,-0.643667,0.284104,0.286555,0.348625,1.814691,-1.351353,-0.473910
1,-5.036009,-12.885333,0.161155,0.592460,3.123534,4.220469,-6.411771,-6.335328,-0.244622,1.346073,...,0.942353,2.938791,1.429883,-2.336344,1.281628,-0.098321,0.582357,0.485792,0.642451,0.613107
2,-9.639157,-6.655898,0.388687,-1.717650,0.300346,3.400769,-7.240785,-1.659405,-0.874005,4.153403,...,-0.413174,-0.023028,-0.025265,1.259838,-0.441360,-0.960094,1.995843,1.097748,0.827182,-1.767840
3,-6.650375,-7.043851,4.104350,-2.342780,3.494658,3.924822,-9.874812,-6.556576,-1.364269,1.153308,...,-0.961236,-1.043815,-0.204508,-1.981150,0.982438,-0.144233,-1.449328,-0.913552,-0.771735,0.304992
4,-10.664524,-10.974133,0.194391,0.453885,2.193088,-3.304663,-8.376592,-4.241146,2.964818,-0.949622,...,0.152957,1.448160,-1.254907,-3.481295,-0.563889,1.529335,0.510399,0.298318,-0.943213,1.149847
5,3.437618,-3.914502,-1.126803,4.037337,-2.513498,1.733343,-8.812785,-4.181726,8.662117,-2.783926,...,-0.561103,0.895975,0.520910,-2.052927,-0.572874,0.179898,-1.105315,-0.758766,1.420641,1.649785
6,-13.986871,-9.610897,9.996927,-3.326522,3.600040,-1.118082,-5.246668,-3.259963,2.397346,-1.800137,...,-1.108424,-1.601396,-1.015069,-1.257553,-0.289895,-0.094534,1.076284,-0.398156,1.522532,0.921088
7,-10.951489,-11.327592,4.453519,-2.693600,1.141457,-0.066174,-9.374738,-6.926078,0.115435,1.788652,...,-0.095260,-0.650402,-0.216118,1.071904,0.805002,0.927228,0.315061,-0.062183,0.984969,0.253087
8,-13.141173,-13.002304,12.334438,-3.001888,2.884243,-0.151152,0.353718,-1.105504,3.598557,0.553618,...,0.555660,-0.094570,0.708782,-0.091800,-0.276942,-0.634866,0.294718,1.020319,-0.566205,0.479066
9,-10.334850,-5.193978,8.168016,-0.392417,2.706182,3.762498,-4.164942,-0.976768,0.544748,0.950181,...,0.278781,-0.830145,-1.152461,1.622025,0.384430,-1.025907,-0.322276,0.541871,0.813516,0.045286


In [6]:
#cria as classes de 0-9
classes = np.array([])
for i in range (2000):
    classes = np.append(classes,int(i/200))
classes[198:203]

array([ 0.,  0.,  1.,  1.,  1.])